**Fewshot**
* 모델이 작업을 수행하거나 매우 제한적인 레이블이 지정된 데이터를 사용하여 새로운 작업에 적응하도록 학습되는 기계 학습 접근 방식

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate

# Chat 모델 초기화
chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

examples = [
{
"question": "What do you know about France?",
"answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
},
{
"question": "What do you know about Italy?",
"answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
},
{
"question": "What do you know about Greece?",
"answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
},
]

example_template = """
    Human: {question}
    AI : {answer}
"""

example_prompt = PromptTemplate.from_template("Human: {question}\n AI:{answer}")

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

chain = prompt | chat

chain.invoke({
    "country": "Korea"
})


AI: 
I know this:
Capital: Seoul
Language: Korean
Food: Kimchi and Bibimbap
Currency: South Korean Won

AIMessageChunk(content='\nAI: \nI know this:\nCapital: Seoul\nLanguage: Korean\nFood: Kimchi and Bibimbap\nCurrency: South Korean Won')

### FewShot 을 이용한 챗봇

In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate

# Chat 모델 초기화
chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

examples = [
{
"country": "France?",
"answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
},
{
"country": "Italy?",
"answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
},
{
"country": "Greece?",
"answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
},
]

# 형식 지정
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "what do you know about {country}?"),
    ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert"),
    example_prompt,
    ("human", "What do you know about {country}?")
])
chain = final_prompt | chat

chain.invoke({
    "country": "America"
})

I know this:
Capital: Washington D.C.
Language: English
Food: Burgers and Hotdogs
Currency: US Dollar

AIMessageChunk(content='I know this:\nCapital: Washington D.C.\nLanguage: English\nFood: Burgers and Hotdogs\nCurrency: US Dollar')

## LengthBasedExampleSelector

In [22]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import example_selector
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]


example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

#prompt.format(country="Brazil")
# 프롬프트 형식화
output = prompt.format(country="Brazil")
print(output)


Human: What do you know about Italy?
AI:
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        

Human: What do you know about Brazil?


### 챌린지
* 영화 이름을 가지고 감독, 주요 출연진, 예산, 흥행 수익, 영화의 장르, 간단한 시놉시스 등 영화에 대한 정보로 답장하는 체인을 만드세요.
* LLM은 항상 동일한 형식을 사용하여 응답해야 하며, 이를 위해서는 원하는 출력의 예시를 LLM에 제공해야 합니다.
* 예제를 제공하려면 FewShotPromptTemplate 또는 FewShotChatMessagePromptTemplate을 사용하세요.


In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate

# Chat 모델 초기화
chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

examples = [
{
"movie": "파묘?",
"answer": """
Here is what I know:
Director: 장재현
a production company: 쇼박스
cast: 최민식, 김고은, 유해진, 이도현
Genre: 공포 
""",
},
{
"movie": "하얼빈?",
"answer": """
I know this:
Director: 홍경표
a production company: CJ ENM
cast: 현빈, 이동욱, 박훈
Genre: 액션, 스릴러, 느와르
""",
},
{
"movie": "보고타?",
"answer": """
I know this:
Director: 김성제
a production company: IDIOPLAN
cast: 송중기, 히의준, 크리스탈 아파리시오
Genre: 스릴러, 범죄물
""",
},
]

# 형식 지정
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{movie}에 대해서 무엇을 알고 있니??"),
    ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 영화 평론가(전문가) 입니다."),
    example_prompt,
    ("human", "{movie}에 대해서 무엇을 알고 있니?")
])
chain = final_prompt | chat

chain.invoke({
    "movie": "기생충"
})

"기생충"은 2019년 개봉한 한국 영화로, 감독은 봉준호입니다. 이 영화는 송강호, 이선균, 조여정, 최우식 등이 출연했고, 코미디, 스릴러, 드라마 장르에 속합니다. "기생충"은 2019년 제 92회 아카데미상에서 최우식이 주연 남우조연상을 수상하며 한국 영화 최초로 아카데미상을 수상한 작품으로 큰 주목을 받았습니다.

AIMessageChunk(content='"기생충"은 2019년 개봉한 한국 영화로, 감독은 봉준호입니다. 이 영화는 송강호, 이선균, 조여정, 최우식 등이 출연했고, 코미디, 스릴러, 드라마 장르에 속합니다. "기생충"은 2019년 제 92회 아카데미상에서 최우식이 주연 남우조연상을 수상하며 한국 영화 최초로 아카데미상을 수상한 작품으로 큰 주목을 받았습니다.')

In [8]:
chain.invoke({
    "movie": "하얼빈"
})

"하얼빈"은 2017년에 개봉한 한국 영화로, 홍경표 감독이 연출하였습니다. 영화의 주요 배우로는 현빈, 이동욱, 박훈 등이 출연했습니다. 이 영화는 액션, 스릴러, 느와르 장르에 속하며, 중국 하얼빈에서 벌어지는 범죄와 복수의 이야기를 다룹니다. 영화는 긴장감 넘치는 전투씬과 강렬한 액션으로 관객들을 매료시키며, 현실적인 연기와 몰입도 높은 스토리로 호평을 받았습니다.

AIMessageChunk(content='"하얼빈"은 2017년에 개봉한 한국 영화로, 홍경표 감독이 연출하였습니다. 영화의 주요 배우로는 현빈, 이동욱, 박훈 등이 출연했습니다. 이 영화는 액션, 스릴러, 느와르 장르에 속하며, 중국 하얼빈에서 벌어지는 범죄와 복수의 이야기를 다룹니다. 영화는 긴장감 넘치는 전투씬과 강렬한 액션으로 관객들을 매료시키며, 현실적인 연기와 몰입도 높은 스토리로 호평을 받았습니다.')

In [9]:
chain.invoke({
    "movie": "미션임파서블"
})

미션 임파서블 시리즈는 톰 크루즈가 주연으로 출연하는 액션 영화 시리즈로, 브라이언 데 팔마 감독이 연출하고 있습니다. 시리즈는 IMF(Impossible Mission Force)라는 비밀 요원 기관을 중심으로 한 다양한 스파이 액션을 다루고 있으며, 톰 크루즈가 주연인 이후에도 제레미 레너, 사이먼 페그, 레베카 퍼거슨 등이 시리즈에 출연하였습니다. 현재까지 6편의 영화가 제작되었고, 7편과 8편이 제작 중이며, 7편은 2022년에 개봉 예정입니다.

AIMessageChunk(content='미션 임파서블 시리즈는 톰 크루즈가 주연으로 출연하는 액션 영화 시리즈로, 브라이언 데 팔마 감독이 연출하고 있습니다. 시리즈는 IMF(Impossible Mission Force)라는 비밀 요원 기관을 중심으로 한 다양한 스파이 액션을 다루고 있으며, 톰 크루즈가 주연인 이후에도 제레미 레너, 사이먼 페그, 레베카 퍼거슨 등이 시리즈에 출연하였습니다. 현재까지 6편의 영화가 제작되었고, 7편과 8편이 제작 중이며, 7편은 2022년에 개봉 예정입니다.')